# **Recommend Jobs**

In [1]:
import requests
import pandas as pd

In [2]:
 def getJob(query):
  url = "https://jsearch.p.rapidapi.com/search"

  querystring = {"query":{query},"page":"1","num_pages":"10"}

  headers = {
    "X-RapidAPI-Key": "6df26507bcmsha4de1a41b8d21fbp1b36d0jsn0e0a8afe6d75",
    "X-RapidAPI-Host": "jsearch.p.rapidapi.com"
  }

  response = requests.get(url, headers=headers, params=querystring)
  result = response.json();
  return result['data'];

In [3]:
def collapse(location):
    return "".join(location.split(", "))

In [4]:
result = getJob("software engineer, india")

In [5]:
original_df = pd.DataFrame.from_dict(result)
original_df = original_df[['job_id',
                  'employer_name',
                  'job_city',
                  'job_state',
                  'job_country',
                  'job_title',
                  'job_apply_link',
                  'job_description',
                  'job_is_remote',
                  'job_required_experience',
                  'job_required_education']]

In [6]:
original_df.dropna(inplace=True)

In [7]:
original_df['location'] = original_df['job_city'] + ", " + original_df['job_state'] + ", " + original_df['job_country']

In [8]:
original_df['location'] = original_df['location'].apply(collapse)

In [9]:
df = pd.DataFrame.from_dict(result)

In [10]:
df = df[['job_id', 'employer_name', 'job_city', 'job_state', 'job_country']]
# df.head()

In [11]:
df['location'] = df['job_city'] + ", " + df['job_state'] + ", " + df['job_country']
# df.head()

In [13]:
df = df.drop(['job_city', 'job_state', 'job_country'], axis='columns')
# df.head()

In [12]:
df.dropna(inplace=True)

In [14]:
df['location'] = df['location'].apply(collapse)

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=100,stop_words='english')

In [16]:
vector = cv.fit_transform(df['location']).toarray()

In [17]:
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
similarity = cosine_similarity(vector)

In [19]:
def recommend(location, original_df):
    index = df[df['location'] == location].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])

    recommended_jobs = set() # Create an empty list to store recommended job details
    for i in distances[1:10]:
        job_id = df.iloc[i[0]].job_id # get the job_id

        #filter the DataFrame to find the job details with matching job_id
        job_details = original_df[original_df['job_id'] == job_id]

        if not job_details.empty and job_id not in recommended_jobs:
          job_apply_link = job_details['job_apply_link'].values[0]
          employer_name = job_details['employer_name'].values[0]
          job_city = job_details['job_city'].values[0]
          job_state = job_details['job_state'].values[0]
          job_country = job_details['job_country'].values[0]
          job_title = job_details['job_title'].values[0]
          job_description = job_details['job_description'].values[0]
          job_required_experience = job_details['job_required_experience'].values[0]
          job_required_education = job_details['job_required_education'].values[0]

          # Append the job details to the recommended_jobs list
          recommended_jobs.add(job_id)

          # Print or return the recommended job details
          # for job in recommended_jobs:
          print("Job ID:",  job_id)
          print("Employer Name:", employer_name)
          print("Job City:", job_city)
          print("Job State:", job_state)
          print("Job Country:", job_country)
          print("Job Title:", job_title)
          # print("Job Description:", job_description)
          # print("Job Experience:", job_required_experience)
          # print("Job Required Education:", job_required_education)
          print("Apply Link:", job_apply_link)
          print("----------------------------------------------------------------------------------------")


In [20]:
# This model will Recommend top 20 jobs in pune maharashtra india
# output: company names which are hiring
recommend("PuneMHIN", original_df)

Job ID: feTIs8ndaArlVQOuAAAAAA==
Employer Name: Alumni Career Site
Job City: Pune
Job State: MH
Job Country: IN
Job Title: Software Engineer III
Apply Link: https://jpmc.fa.oraclecloud.com/hcmUI/CandidateExperience/en/sites/CX_2001/job/210476519
----------------------------------------------------------------------------------------
Job ID: 6VVjd6PX-FjvLoAQAAAAAA==
Employer Name: Global Payments
Job City: Pune
Job State: MH
Job Country: IN
Job Title: Associate Lead Software Engineer
Apply Link: https://jobs.globalpayments.com/en/jobs/r0046377/associate-lead-software-engineer/
----------------------------------------------------------------------------------------
Job ID: XtYlze7JgVHUjqzVAAAAAA==
Employer Name: Mastercard
Job City: Pune
Job State: MH
Job Country: IN
Job Title: Software Engineer I
Apply Link: https://careers.mastercard.com/us/en/job/R-201744/Software-Engineer-I
----------------------------------------------------------------------------------------
Job ID: rYfoaDYMeCMaqk